In [1]:
#Interfaz
!pip install PySimpleGUI

In [2]:
#Mock arbol
class Arbol:
    pass

class Alineador:
    pass

arbol = Arbol()
Alineacion = Alineador()

In [3]:
#Mapa - libreria
!pip install geopy
!pip install folium

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="TP FINAL Bioinformatica")

In [4]:
#Mapa - Modelo
import folium

class Mapa:
        
    def mostrarMapa(self, nombres_de_ubicaciones):
        points = []
        my_map = folium.Map(
            zoom_start=8
        )
        for nombre_ubicacion in nombres_de_ubicaciones:
            ubicacion = geolocator.geocode(nombre_ubicacion)
            point = (ubicacion.latitude, ubicacion.longitude)
            points.append(point)
            folium.Marker(
              location=[point[0], point[1]],
              popup='',
              icon=folium.Icon(color='red', icon='cloud')
            ).add_to(my_map)
        folium.PolyLine(points).add_to(my_map)
        display(my_map)

mapa = Mapa()

In [ ]:
import PySimpleGUI as sg

newick = ''
fasta = ''
lista_de_nombre_de_ubicaciones = ["Londres", "Paris", "Amsterdam", "Las Vegas", "Tokyo", "Buenos Aires", "San Pablo", "Tijuana", "Medellin", "Texas", "Nueva Orleans"]

layoutSubirArchivo = [  [sg.Text('Filename')],
            [sg.Input(), sg.FileBrowse()], 
            [sg.OK(), sg.Cancel()]]

sg.theme('DarkAmber')   # Add a touch of color
# All the stuff inside your window.
layout = [  [sg.Text('MENU Pongamosle un lindo mensaje')],
            [sg.Button('Subir Fasta')],
            [sg.Text('O ingrese manualmente la secuencia.')],
            [sg.InputText()],
            [sg.Button('Cargar')],
            [sg.Button('Alinear Secuencias')],
            [sg.Button('Armar Arbol')],
            [sg.Button('Mostrar Mapa')],
         ]

window = sg.Window('Trabajo Final Bioinformatica', layout)
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED: 
        break
    elif event == 'Subir Fasta':
        event, values = sg.Window('Seleccione un archivo tipo Fasta', layoutSubirArchivo).read(close=True)
    elif event == 'Alinear Secuencias':
        alineacion.alinear()
    elif event == 'Armar Arbol':
        arbol.armarArbol()
    elif event == 'Mostrar Mapa':
        mapa.mostrarMapa(lista_de_nombre_de_ubicaciones)

window.close()



HOOLAAAAAAAAAAAAAAaa
